In [ ]:
import json
import torch

from world_model.ecosystem_graph import EcosystemGraph
from world_model.dynamics import EcosystemDynamics
from world_model.constraints import EcologicalConstraints

from models.embeddings import StateEmbedding
from models.transformer_policy import TransformerPolicy
from models.value_model import ValueModel

from agents.simulator import Simulator
from agents.planner import AgentPlanner
from mcp.context_builder import ContextBuilder
from mcp.human_feedback import HumanFeedbackLoop

In [ ]:
with open("../data/toy_ecosystem.json") as f:
    data = json.load(f)

graph = EcosystemGraph()
for s in data["species"]:
    graph.add_species(**s)

for i in data["interactions"]:
    graph.add_interaction(i["src"], i["dst"], i["strength"])

In [ ]:
dynamics = EcosystemDynamics(graph)
constraints = EcologicalConstraints()

embedder = StateEmbedding(input_dim=2, embed_dim=16)
policy = TransformerPolicy(embed_dim=16, num_heads=2, num_layers=2, action_dim=2)
value_model = ValueModel(input_dim=2)

simulator = Simulator(dynamics, constraints)
planner = AgentPlanner(
    policy,
    value_model,
    simulator,
    graph.species_names()
)

In [ ]:
state = graph.state_vector().unsqueeze(0)
embedded = embedder(state).unsqueeze(1)

intervention, score = planner.plan(embedded)

feedback = HumanFeedbackLoop().collect(intervention, score)

intervention, score, feedback